# OpenMDAO Example

We will reuse the [Sellar problem](https://openmdao.org/newdocs/versions/latest/basic_user_guide/multidisciplinary_optimization/sellar.html) from the OpenMDAO documention. It is an example of two-discipline optimization problem with feedback coupling. 


### Libraries 

OpenMDAO must me installed: 

In [ ]:
# %conda install openmdao

Channels:
 - conda-forge
Platform: osx-arm64
Solving environment: done

# All requested packages already installed.


Note: you may need to restart the kernel to use updated packages.


In [2]:
import ipysensitivityprofiler as isp
import openmdao.api as om 

In [3]:
from _openmdao_profiler import openmdao_profiler
from _openmdao_models import SellarMDA, SellarView

### MDAO Problem Instantiation

In [4]:
prob = om.Problem(model=SellarMDA(num_nodes=1)).setup()  

In [9]:
om.n2(prob)

### Optimization



In [5]:
prob.driver = om.ScipyOptimizeDriver()
prob.driver.options['optimizer'] = 'SLSQP'
# prob.driver.options['maxiter'] = 100
prob.driver.options['tol'] = 1e-8

prob.model.add_design_var('x', lower=0, upper=10)
prob.model.add_design_var('z1', lower=0, upper=10)
prob.model.add_design_var('z2', lower=0, upper=10)
prob.model.add_objective('obj')
prob.model.add_constraint('con1', upper=0)
prob.model.add_constraint('con2', upper=0)

# Ask OpenMDAO to finite-difference across the model to compute the gradients for the optimizer
prob.model.approx_totals()

prob.setup()
prob.set_solver_print(level=0)
prob.run_driver()

print('minimum found at')
print(prob.get_val('x')[0])
print(prob.get_val('z1'))
print(prob.get_val('z2'))

print('minumum objective')
print(prob.get_val('obj')[0])

Optimization terminated successfully    (Exit mode 0)
            Current function value: 3.162086954524598
            Iterations: 11
            Function evaluations: 11
            Gradient evaluations: 11
Optimization Complete
-----------------------------------
minimum found at
5.908901840045999e-16
[7.85470778e-15]
[4.39440972]
minumum objective
3.162086954524598


### Local Sensitivity Visualization

We setup a separate `Group` that does not include a solver, which we will use to understand local sensitivities in the vicininity of the optimum. 

In [7]:
profiler = openmdao_profiler(
    problem=om.Problem(model=SellarView()).setup(),
    inputs=[
        ("x", 0, 10, None), 
        ("z1", 0, 3, None), 
        ("z2", 0, 10, None), 
        ("y1", 0, 10, None), 
        ("y2", 0, 10, None), 
    ],
    outputs=[
        ("Residual1", -5, 5, None), 
        ("Residual2", -5, 5, None), 
        ("obj", -5, 50, None), 
        ("con1", -5, 5, None), 
        ("con2", -50, 0, None), 
    ],
    defaults=[  # pass converged inputs
        ("x", prob.get_val("x"), None),
        ("z1", prob.get_val("z1"), None),
        ("z2", prob.get_val("z2"), None),
        ("y1", prob.get_val("y1"), None),
        ("y2", prob.get_val("y2"), None),
    ],
    resolution=100,
    width=200,
    height=None,
)
profiler.view

View(children=(GridspecLayout(children=(Figure(axes=[Axis(label='x', num_ticks=3, scale=LinearScale(max=np.flo…

In [8]:
profiler.controller

Controller(children=(FloatSlider(value=5.908901840045999e-16, description='x', max=10.0), FloatSlider(value=7.…